* sauce: https://towardsdatascience.com/understanding-how-schools-work-with-canonical-correlation-analysis-4c9a88c6b913
* see also: https://online.stat.psu.edu/stat505/lesson/13/13.5

In [2]:
import numpy as np
import pandas as pd
import rcca

In [3]:
d0 = pd.read_csv("/Users/don/Documents/scratch/pyrcca/kaggle-nyc-schools-2016.csv")
d0 = d0[['Rigorous Instruction %',
      'Collaborative Teachers %',
     'Supportive Environment %',
       'Effective School Leadership %',
   'Strong Family-Community Ties %',
    'Trust %','Average ELA Proficiency',
       'Average Math Proficiency']]
d0 = d0.dropna()

In [5]:
X = d0[['Rigorous Instruction %',
      'Collaborative Teachers %',
     'Supportive Environment %',
       'Effective School Leadership %',
   'Strong Family-Community Ties %',
    'Trust %'
      ]]
Y = d0[['Average ELA Proficiency',
       'Average Math Proficiency']]
print(X.shape, Y.shape)

(1217, 6) (1217, 2)


In [6]:
# Standardise the data
for col in X.columns:
    X[col] = X[col].str.strip('%')
    X[col] = X[col].astype('float')

from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=True, with_std=True)
X_sc = sc.fit_transform(X)
Y_sc = sc.fit_transform(Y)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [12]:
nComponents = 2 # min(p,q) components
cca = rcca.CCA(kernelcca = False, reg = 0., numCC = nComponents,)
# train on data
cca.train([X, Y])
print('Canonical Correlation Per Component Pair:',cca.cancorrs)
print('% Shared Variance:',cca.cancorrs**2)


Training CCA, kernel = None, regularization = 0.0000, 2 components
Canonical Correlation Per Component Pair: [0.35586915 0.27781991]
% Shared Variance: [0.12664285 0.0771839 ]


We should take into account that it is not highly recommended to rely on weights when interpreting individual variable contribution to covariates.

1. Weights are subject to variability from one sample to another.
2. Weights can be highly affected by multicollinearity (which is quite common for same-context variable groups).

Relying on canonical loadings instead is a more common practice.


In [16]:
## Canonical Loadings
print('Corr of CVX, Math Score:',np.corrcoef(cca.comps[0][:,0],Y.values[:,0])[0,1])
print('Corr of CVX, ELA Score:',np.corrcoef(cca.comps[0][:,0],Y.values[:,1])[0,1])


Corr of CVX, Math Score: -0.3933845694445854
Corr of CVX, ELA Score: -0.4323220140505672


In [17]:
np.corrcoef(cca.comps[0][:,0],Y.values[:,0])

array([[ 1.        , -0.39338457],
       [-0.39338457,  1.        ]])

In [9]:
## Canonical Covariates

# CVX 
cca.comps[0]
# First CV for X 
cca.comps[0][:,0]
# Second CV for X
cca.comps[0][:,1]

# CVY
cca.comps[1]
# First CV for Y
cca.comps[1][:,0]
# Second CV for Y 
cca.comps[1][:,1]

array([-0.00760554, -0.01886755, -0.00862955, ..., -0.00883273,
       -0.05742338, -0.02446945])